In [1]:
from __future__ import division
import pandas as pd
import numpy as np
from TumorTruths import TumorTruths
from scipy.stats import mode
from sklearn.metrics import f1_score
from sklearn import cluster
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
svnCalls = pd.read_csv('SNVCalls_IS4.txt', delimiter='\t', dtype={'CHROM':pd.np.str})
truths = TumorTruths('synthetic.challenge.set4.tumour.25pctmasked.truth.vcf')

In [7]:
resultsArray = truths.GetTruths(svnCalls)
print len(svnCalls.columns)

226


In [4]:
mostCommonPredictions = mode(svnCalls[svnCalls.columns[3:-1]], axis=1)[0]
f1_score(mostCommonPredictions, resultsArray)

0.83674707935062975

In [5]:
num_clusters = 5
k_means = cluster.KMeans(n_clusters=num_clusters)
k_means.fit(svnCalls[svnCalls.columns[3:-1]].T)

MemoryError: 

In [ ]:
def TransformEuclideanPointsToPredictions(point):
    return 1 if point >= 0.5 else 0

toPredictions = np.vectorize(TransformEuclideanPointsToPredictions) 
clusterPredictions = np.zeros(len(resultsArray))
for i in range(0, num_clusters):
    weight = len([1 for num in k_means.labels_  if num == i ])
    cosineSimilarities = cosine_similarity(svnCalls[svnCalls.columns[3:-1]].T, k_means.cluster_centers_[i])
    bestIndex = np.argmax(cosineSimilarities)
    representativeCaller = svnCalls[svnCalls.columns[3 + bestIndex]]
    clusterPredictions += weight * representativeCaller

clusterPredictions /= len(k_means.labels_)
clusterPredictions = toPredictions(clusterPredictions)

In [ ]:
f1_score(clusterPredictions, resultsArray)